In [125]:
using LaTeXStrings
using StatsPlots
using PrettyTables
using Distributions
using StatsBase
using Corpuscles
using UnicodeFun
using FastJet
using CxxWrap: StdVector
using LCIO
using LinearAlgebra
using JLD2
#pyplot()
#gr()
#theme(:gruvbox_dark)

In [126]:
fileList_ss = filter(s->occursin(r"E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8\.pR0.3\..*_DST.slcio", s), readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

fileList_bb = filter(s->occursin(r"E250_SetA.Pmumuh2bb.Gwhizard-2_84.eL0.8\.pR0.3\..*_DST.slcio", s), readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

fileList_cc = filter(s->occursin(r"E250_SetA.Pmumuh2cc.Gwhizard-2_84.eL0.8\.pR0.3\..*_DST.slcio", s), readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

fileList_gg = filter(s->occursin(r"E250_SetA.Pmumuh2gg.Gwhizard-2_84.eL0.8\.pR0.3\..*_DST.slcio", s), readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

print("ss has ")
print(length(fileList_ss))
print(" files, bb has ")
print(length(fileList_bb))
print(" files, cc has ")
print(length(fileList_cc))
print(" files, gg has ")
print(length(fileList_gg))
print(" files")
sampleList=[fileList_ss, fileList_bb, fileList_cc, fileList_gg]

ss has 100 files, bb has 100 files, cc has 100 files, gg has 100 files

4-element Array{Array{String,1},1}:
 ["/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.0_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.10_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.11_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.12_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.13_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.14_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.15_DST.slcio", "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.16_DST.slcio", "/nfs/dust/ilc/user/jstrube

In [127]:
@load "FalsePosIndex"
@load "NoParents"
@load "RecoPDG"
@load "DifferenceList"
@load "nEvents"
@load "Purity"
@load "Efficiencies"

1-element Array{Symbol,1}:
 :efficiencyList

In [60]:
function jetConstituentP(constituent)
    a=FastJet.px(constituent)::Float64
    b=FastJet.py(constituent)::Float64
    c=FastJet.pz(constituent)::Float64
    return (a,b,c)
end

function leadingParticleIndex(jet)
    momentumList = []
    highestMomentum = 0
    nCon = 0
    index = 0
    for c in constituents(jet)
        nCon+=1
        p=jetConstituentP(c)
        if dot(p, p)>highestMomentum
            highestMomentum=dot(p, p)
            index=nCon
        end
    end
    return index
end

function invMassSum(particle_1, particle_2)
    a = sqrt((getEnergy(particle_1)+getEnergy(particle_2))^2-
        dot(getMomentum(particle_1)+getMomentum(particle_2),
            getMomentum(particle_1)+getMomentum(particle_2)))
    return a
end

function recoil_mass(μ₁, μ₂)
    E_initial = 250.0  #can update to make it accurate for the event's flucation, with p as well?
    px_initial = 0.0
    recoilE = E_initial - getEnergy(μ₁) - getEnergy(μ₂)
    recoilP = [px_initial, 0, 0] - getMomentum(μ₁) - getMomentum(μ₂)
    return sqrt(recoilE^2 - sum(recoilP.^2))
end

recoil_mass (generic function with 1 method)

In [134]:
function zMuonIndices(event, threshold, MCTruth=false) #for certain events it returns an empty candidatePairs
    
    iParticle = 0
    posMuonList = []
    negMuonList = []
    
    if MCTruth==false
        collection = getCollection(event, "PandoraPFOs")
        for particle in collection
            iParticle += 1
            if abs(getType(particle))==13 #&& getGeneratorStatus(particle)==1 #assuming all pfos are final state
                if getCharge(particle)==1
                    push!(posMuonList, (particle, iParticle))
                elseif getCharge(particle)==-1
                    push!(negMuonList, (particle, iParticle))
                end
            end
        end
        
    elseif MCTruth==true
        collection = getCollection(event, "MCParticle")
        for particle in collection
            iParticle += 1
            if abs(getPDG(particle))==13 && getGeneratorStatus(particle)==1
                if getCharge(particle)==1
                    push!(posMuonList, (particle, iParticle))
                elseif getCharge(particle)==-1
                    push!(negMuonList, (particle, iParticle))
                end
            end
        end      
    end

    
    candidatePair=[]
    
    candidateDiff=91.187
    for item_pos in posMuonList    
        posParticle = item_pos[1]
        for item_neg in negMuonList
            negParticle = item_neg[1]
            if abs(91.187-invMassSum(posParticle, negParticle))<abs(candidateDiff)
                candidateDiff=91.187-invMassSum(posParticle, negParticle)
                candidatePair=[item_pos, item_neg]
            end
        end
    end
    
    if abs(candidateDiff)>threshold
        indices=[0,0]
    elseif length(candidatePair)>0
        indices = [candidatePair[1][2], candidatePair[2][2]]
    end
    return indices
end

function trueZMuonFinder(event)
    
    nZ=0
    muons=[]
    mcCollection = getCollection(event, "MCParticle")
    for (iParticle, particle) in enumerate(mcCollection)
        if abs(getPDG(particle))==13 && getGeneratorStatus(particle)==1
            muonParent=false
            for parent in getParents(particle)
                if abs(getPDG(parent))==13 
                    push!(muons, parent)
                end
            end
        end
    end
    return muons
end
        

trueZMuonFinder (generic function with 1 method)

To Do:\
Normalize plots?\
Dont use getRelatedTooObjects for Reco - for pdg use abs(getType(pfo)==13)\
Check efficiencies of only one muon\
Dont require 2 MC Muons for efficiency calc\
Look at efficiencies with a difference z threshold\
Create a function for getting true z daughters\

In [137]:
@time for sample in [fileList_ss]
    iFile=0
    recoPDG=[]
    falsePosEventIndexList=[]
    zMuonPurityList=[]
    perFilePurity=[]
    noParentIndexList=[]
    perFileNoParents=[]
    perSampleRecoMuonMassList = []
    perSampleMcMuonMassList = []
    perSampleDifferenceList = []
    perSampleRecoilMass = []
    nEvents=0
    nZ=0
    nEfficient=0
    baseEfficient=0
    for FILENAME in sample
        iFile+=1
        if iFile>5
            break
        end
        LCIO.open(FILENAME) do reader
            iEvent=0
            nFalsePos=0
            nNoParents=0
            falsePos=true
            missingRecoMuon=false
            for event in reader
                iEvent+=1
                nEvents+=1
                if iEvent>200
                    break
                end   
                RP_zMuonIndex=zMuonIndices(event, 50)
            
                mcMuons=trueZMuonFinder(event)
                if length(mcMuons)>0
                    nZ+=1
                end
                
                rpCollection = getCollection(event, "PandoraPFOs")
                recoLinks = getCollection(event, "RecoMCTruthLink")
                rel = LCIO.LCRelationNavigator(recoLinks)
                #mcLinks = getCollection(event, "MCTruthRecoLink")
                #mcRel = LCIO.LCRelationNavigator(mcLinks)
                recoMuons=[]
    #            if iEvent==5 && iFile==2
     #               for (iParticle, particle) in enumerate(rpCollection)
      #                  mcp = getRelatedToObjects(rel, particle)
       #                 if length(mcp)>0
        #                    MCParticle=mcp[1]
         #               else
          #                  continue
           #             end
            #            print(getPDG(MCParticle))
             #           print(" ")
              #      end
               # end
                
                for (iParticle, particle) in enumerate(rpCollection)
                    if iParticle in RP_zMuonIndex
                        push!(recoMuons, particle)
                    end
                end
                
                for rpMuon in recoMuons
                    mcp = getRelatedToObjects(rel, rpMuon)
                    if length(mcp)>0
                        MCParticle=mcp[1]
                    else
                        continue
                    end
                    for MCP in mcp
                        for parent in getParents(MCP)
                            if abs(getPDG(parent))==13
                                falsePos=false
                            end
                        end
                    end
                end
                
                for mcMuon in mcMuons
                    RP=getRelatedFromObjects(rel, mcMuon)
                    if length(RP)==0
                        if iEvent==2 && iFile==2
                            print(":(")
                        end
                        missingRecoMuon=true
                    end
                end
            
                
                if length(recoMuons)==2
                    nEfficient+=1
                    push!(perSampleRecoilMass, recoil_mass(recoMuons[1], recoMuons[2]))
                    if length(mcMuons)==2
                        push!(perSampleDifferenceList, 
                            invMassSum(recoMuons[1], recoMuons[2])-invMassSum(mcMuons[1], mcMuons[2]))
                    end
                end
                
                if falsePos==true
                   nFalsePos+=1
                end
                if missingRecoMuon==false
                    baseEfficient+=1
                end
            end
            push!(perFilePurity, 1-nFalsePos/iEvent)
            push!(perFileNoParents, nNoParents/iEvent)
        end
    end
    print(mean(perFilePurity))
    print(" ")
    print(mean(perFileNoParents))
    print(" ")
    print(nEfficient/nEvents)
    print(" ")
    print(baseEfficient/nEvents)
    print(" ")
    print(length(perSampleRecoilMass)/length(perSampleDifferenceList))
end


:(:(0.9980099502487562 0.0 0.6686567164179105 0.0 1.0  3.957511 seconds (2.11 M allocations: 65.710 MiB)


In [14]:
print(perSamplePurity)

Dict{Any,Any}("ss" => 1.0,"bb" => 1.0,"gg" => 1.0,"cc" => 1.0)

In [15]:
print(perSampleNoParents)

Dict{Any,Any}("ss" => 0.0,"bb" => 0.0,"gg" => 0.0,"cc" => 0.0)

In [16]:
print(efficiencyList)

Dict{Any,Any}("ss" => 0.6253989637305699,"bb" => 0.626544502617801,"gg" => 0.62194,"cc" => 0.6249119170984456)

In [112]:
0.9980099502487562 0.0 0.6746268656716418 0.9950248756218906  4.151472 seconds (2.12 M allocations: 64.998 MiB, 2.15% gc time)

LoadError: syntax: extra token "0.0" after end of expression

In [ ]:
0.9980099502487562 0.0 0.690547263681592 0.9950248756218906  4.396963 seconds (2.34 M allocations: 72.042 MiB)